In [1]:
# If cloning is needed -- i.e., if hangman.py and word counts are not already
# available in file, you can run this
!git clone https://github.com/ucsd-cse150a-w25/hw1.git
!cp hw1/hangman.py hangman.py
!cp hw1/hw1_word_counts_05.txt hw1_word_counts_05.txt

Cloning into 'hw1'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 55 (delta 26), reused 28 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (55/55), 723.52 KiB | 7.31 MiB/s, done.
Resolving deltas: 100% (26/26), done.


In [ ]:
from hangman import hangman_game

# Play the game by yourself
hangman_game()




  +---+
      |
      |
      |
     ===

Word: _ _ _ _ _
Tried letters: 


In [ ]:
import random

def random_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Random inference for playing hangman. This should be a simple method which returns a letter
    that is NOT in letters_tried but any other letter, at random.

    Hint: use the random.choice method
    '''
    # TODO: Implement random inference code to guess hangman
    all_letters = set(chr(i) for i in range(ord('A'), ord('Z') + 1))
    available_letters = list(all_letters - letters_tried)
    print(f"Available Letters: {available_letters}")  # Debugging print
    return random.choice(available_letters) if available_letters else None

hangman_game(random_inference)

In [ ]:
def bayesian_inference(
    letters_tried: set[str],
    word_pattern: list[str],
    word_counts: dict[str, int]
) -> str:
    '''
    Bayesian inference method for playing hangman. The parameters given are as follows:

    - letters_tried (set[str]): A set of strings which consist of all the letters that have already
        been tried. For example, if 'A', 'E' has been guessed, `letters_tried = {'A', 'E'}`
    - word_pattern (list[str]): A list of single characters that describe the current guess state.
        For example, if the hangman state is _AB__, `word_pattern = ['_', 'A', 'B', '_', '_']`
    - word_counts (dict[str, int]): The word counts dictionary which contains all possible 5 letter
        words in our hangman game and their respective counts. For example, a key value pair could
        be 'AARON': 413.

    Return type: a single character string as your next best guess.
    '''
    # TODO: Implement inference code to play hangman optimally
    letter_probabilities = {}
    for word, count in word_counts.items():
        # Ensure the word can still be a candidate based on the current pattern and tried letters
        if all((p == '_' or p == word[i]) for i, p in enumerate(word_pattern)):
            for i, char in enumerate(word):
                if word_pattern[i] == '_' and char not in letters_tried:
                    letter_probabilities[char] = letter_probabilities.get(char, 0) + count

    # Debugging: print possible letter probabilities
    print("Letter Probabilities:", sorted(letter_probabilities.items(), key=lambda x: x[1], reverse=True)[:5])

    if letter_probabilities:
        max_letter = max(letter_probabilities, key=letter_probabilities.get)
        return max_letter
    else:
        # Fallback to random choice if no probabilities could be calculated
        return random_inference(letters_tried, word_pattern, word_counts)


# Run the game
hangman_game(bayesian_inference)

In [21]:
from typing import Optional, Callable
from tqdm import tqdm
# from tqdm import tqdm

def benchmark(
    inference: Optional[Callable],
    num_runs: int = 1000,
    seed: int = 0
) -> None:
    '''
    Benchmark method for testing out the bayesian inference method. The parameters given are:

    - inference: The function that should match the bayesian_inference() method above.
    - seed: The seed to pass into the hangman_game function.

    Return type: none.

    Print the accuracy out of num_runs. If the function throws an error, it should count as a fail.
    '''
    # TODO: Implement benchmark testing for a given function
    # Hint: use `random.seed(seed)` ONCE at the start
    # Hint: pass `interactive = False` into the hangman_game function to run faster without outputs.
    # Optional: use the `tqdm` module to keep track of the evaluation progress.
    random.seed(seed)
    success_count = 0
    failures = 0

    for _ in tqdm(range(num_runs), desc="Running Benchmark"):
        try:
            result = hangman_game(inference, interactive=False)
            if result == 'WIN':
                success_count += 1
        except Exception as e:
            failures += 1
            print(f"Error during game: {e}")

    accuracy = (success_count / num_runs) * 100
    print(f"Accuracy: {accuracy:.2f}%, Failures: {failures}/{num_runs}")


benchmark(random_inference)
benchmark(bayesian_inference)

Running Benchmark:   1%|▏         | 14/1000 [00:00<00:07, 132.76it/s]

Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'S', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 

Running Benchmark:   4%|▍         | 41/1000 [00:00<00:07, 123.39it/s]

Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'P', 'U', 'J', 'O', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'P', 'U', 'J', 'O', 'K', 'Y', 'Q', 'T', 'M', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'U', 'J', 'O', 'K', 'Y', 'Q', 'T', 'M', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'F', 'N', 'U', 'O', 'K', 'Y', 'Q', 'T', 'M', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'E', 'V', 'F', 'N', 'O', 'Q', 'T', 'W', 'S', 'Z', 'L', 'G', 'U', 'K', 'Y', 'M', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 

Running Benchmark:   7%|▋         | 66/1000 [00:00<00:07, 117.30it/s]

...

Running Benchmark:  87%|████████▋ | 869/1000 [00:07<00:01, 122.09it/s]

Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'D', 'R']
Available Letters: ['H', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'D', 'R']
Available Letters: ['H', 'L', 'W', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'T', 'M', 'X', 'D', 'R']
Available Letters: ['H', 'E', 'V', 'I', 'A', 'F', 'N', 'O', 'B', 'T', 'W', 'L', 'P', 'U', 'J', 'K', 'Y', 'M', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 

Running Benchmark:  90%|████████▉ | 896/1000 [00:07<00:00, 121.77it/s]

Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'I', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'H', 'Z', 'L', 'W', 'G', 'E', 'V', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'A', 'F', 'N', 'P', 'U', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'A', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'M', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'A', 'F', 'N', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'Z', 'L', 'W', 'G', 'E', 'V', 'A', 'F', 'P', 'J', 'O', 'B', 'K', 'Y', 'Q', 'T', 'X', 'S', 'D', 'R']
Available Letters: ['C', 'E', 'V', 'A', 'F', 'O', 'B', 'Q', 'T', 'W', 'Z', 'L', 'G', 'P', 'J', 'K', 'Y', 'X', 'D', 'R']
Available Letters: ['C', 'E', 'V', 'A', 'F', 'O', 'B', 

KeyboardInterrupt: 